# xarray for pandas users

Key selling points for xarray for me:

1. Inference data from probabilistic programming languages like `pymc` and `numpyro` return results in `xarray` format, so it's useful to understand how to navigate this structure
2. I often need to have a data structure that can accommodate data of variable lengths and potentially more than 2 dimensions, and `xarray` solves this problem for me and I don't need to create a custom data structure.



I assume the reader is familiar with `pandas` and I will be translating concepts back and forth.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

## DataArray

It may be helpful to create a `DataArray` from scratch:

In [14]:
data = np.random.rand(4, 3)
times = pd.date_range("2000-1-1", periods=4)
locs = ['CA', 'OR', 'WA']
foo = xr.DataArray(data=data, coords=[times, locs], dims=['time', 'location'])
foo

<xarray.DataArray (time: 4, location: 3)>
array([[0.39910887, 0.27300257, 0.02524338],
       [0.79183532, 0.26428246, 0.26890157],
       [0.77980469, 0.83571996, 0.36486999],
       [0.0398997 , 0.95004422, 0.5584906 ]])
Coordinates:
  * time      (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * location  (location) <U2 'CA' 'OR' 'WA'

In [24]:
foo.dims

('time', 'location')

Here we a (4,3) array in which each row is from a time and each column is a location. The dimensions in this array are `(time, location)`, and dimensions are named indices/coordinates.

The terminology is...confusing. We see the DataArray has Coordinates, Indexes, and Dimensions `(time, location)`. 

## Coordinates vs Dimensions

Recall that pandas has a built in row index (and supports multiple indices) - these are your `coords` or Coordinates in an `xarray.DataArray`. `xarray` takes it a little further and requires `dims` (Dimensions) which are names for each of your indices.

You can have more coordinates than dimensions. 

* Dimension Coordinate: Usually an Indexed Coordinate - so repeated index labels are a no-no, just like in a pandas index.
* Non-Dimension Coordinate: named Coordinate but not in `dims`. Can be 1-D or multi-dimensional. 
* Indexed Coordinate: Means that the coordinate label can be used for subsetting (i.e. a SQL SELECT statement). Can be 1 or more dimensions

Coordinates are 

In [4]:
foo.coords

Coordinates:
  * time     (time) datetime64[ns] 2000-01-01 2000-01-02 2000-01-03 2000-01-04
  * space    (space) <U2 'CA' 'OR' 'WA'

Coordinates are fixed, and label the points along the dimensions

What would a price table look like?

In [5]:
prices = [0.99,2.99,6.99]
products = ['a', 'b', 'c']
df = pd.DataFrame({'price':prices, 'product_name':products})
df

,price,product_name
0,0.99,a
1,2.99,b
2,6.99,c


In [6]:
# as a data array
da = xr.DataArray(data=prices,
                  coords={'product_name':products},
                  name='price')
da

<xarray.DataArray 'price' (product_name: 3)>
array([0.99, 2.99, 6.99])
Coordinates:
  * product_name  (product_name) <U1 'a' 'b' 'c'

We can use `.assign_coords(coord_name=("dimension_name_to_index_on", [values]))` to create a new coord

In [39]:
da = da.assign_coords(product_grouping=("product_name", [1,2,2]))
da

<xarray.DataArray 'price' (product_name: 3)>
array([0.99, 2.99, 6.99])
Coordinates:
  * product_name      (product_name) <U1 'a' 'b' 'c'
    product_grouping  (product_name) int64 1 2 2

In [40]:
da.coords

Coordinates:
  * product_name      (product_name) <U1 'a' 'b' 'c'
    product_grouping  (product_name) int64 1 2 2

We see this is a non-indexed, non-dimension coord that we just made. 

Next, how would you make a sales `DataArray`?

In [15]:
sales = pd.DataFrame({'product_name':['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'], 
                      'sales':[100, 85, 90, 50,45,55, 60, 65, 70],
                      'week':[1,2,3,1,2,3,1,2,3]})
sales

,product_name,sales,week
0,a,100,1
1,a,85,2
2,a,90,3
3,b,50,1
4,b,45,2
5,b,55,3
6,c,60,1
7,c,65,2
8,c,70,3


In [16]:
sales.to_xarray()

<xarray.Dataset>
Dimensions:       (index: 9)
Coordinates:
  * index         (index) int64 0 1 2 3 4 5 6 7 8
Data variables:
    product_name  (index) object 'a' 'a' 'a' 'b' 'b' 'b' 'c' 'c' 'c'
    sales         (index) int64 100 85 90 50 45 55 60 65 70
    week          (index) int64 1 2 3 1 2 3 1 2 3

We actually want product_name and week to be `dims`, while `sales` should be our sole variable. We should instead reshape the sales data and make a `DataArray`

In [9]:
sales_pivoted = sales.pivot(index='week', values='sales',columns='product_name')
sales_pivoted

product_name,a,b,c
week,,,
1,100,50,60
2,85,45,65
3,90,55,70


Now we can make our dataarray with `week` on rows and `product_name` on columns:

In [10]:
sales_DA = xr.DataArray(sales_pivoted, name='sales')
sales_DA

<xarray.DataArray 'sales' (week: 3, product_name: 3)>
array([[100,  50,  60],
       [ 85,  45,  65],
       [ 90,  55,  70]])
Coordinates:
  * week          (week) int64 1 2 3
  * product_name  (product_name) object 'a' 'b' 'c'

## DataSets

Then, finally we can merge our price and sales `DataArray`s together into a combined `DataSet`. It's simple with `xr.merge` and it will automatically join:

In [17]:
salesDS = xr.merge([da, sales_DA])
salesDS

<xarray.Dataset>
Dimensions:       (product_name: 3, week: 3)
Coordinates:
  * product_name  (product_name) <U1 'a' 'b' 'c'
  * week          (week) int64 1 2 3
Data variables:
    price         (product_name) float64 0.99 2.99 6.99
    sales         (week, product_name) int64 100 50 60 85 45 65 90 55 70

explicitly product_name as a coordinate and an index, and week is an index as well. We can inspect the result as follows:

In [18]:
salesDS.to_dataframe()

price  sales
product_name week              
a            1      0.99    100
             2      0.99     85
             3      0.99     90
b            1      2.99     50
             2      2.99     45
             3      2.99     55
c            1      6.99     60
             2      6.99     65
             3      6.99     70

It uses the coordinates as a multiindex, and each variable is a column. Nifty!

Which makes sense, these coordinates/dimensions must be unique. But what if you have multiple coordinates, not all of which are dimensions?

## Subsetting

Use the `.where` method will subset the data, and set `drop=True` to drop all the rows that returned `np.nan`

In [21]:
# Without drop
salesDS.where(salesDS.week > 2).price.to_dataframe()

price
product_name week       
a            1       NaN
             2       NaN
             3      0.99
b            1       NaN
             2       NaN
             3      2.99
c            1       NaN
             2       NaN
             3      6.99

In [22]:
# with drop
salesDS.where(salesDS.week > 2, drop=True).price.to_dataframe()

,,price
product_name,week,
a,3,0.99
b,3,2.99
c,3,6.99
